In [1]:
from transformer_lens import HookedTransformer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "gpt2"
model = HookedTransformer.from_pretrained(model_name, device = device)

/Users/wendysun/Desktop/sae-rm/stats/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/wendysun/Desktop/sae-rm/stats/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2 into HookedTransformer


In [2]:
from sparsifiers import SAE
from huggingface_hub import hf_hub_download

layer = 6
# e2e_name = "https://huggingface.co/apollo-research/e2e-saes-gpt2/blob/main/downstream_similar_ce_layer_6.pt"
downstream_name = f"downstream_similar_ce_layer_{layer}.pt"
# downstream_name = f"downstream_similar_l0_layer_{layer}.pt"
local_name = f"local_similar_ce_layer_{layer}.pt"
# local_name = f"local_similar_l0_layer_{layer}.pt"

activation_name = f"transformer.h.{layer}"
model_id = "apollo-research/e2e-saes-gpt2"

In [3]:
def load_sae(model_id, hh_file_location):
    ae_download_location = hf_hub_download(repo_id=model_id, filename=hh_file_location)

    sae_list = torch.load(ae_download_location)

    # Rename the keys in the state dictionary
    def rename_keys(state_dict):
        renamed_state_dict = {}
        for key, value in state_dict.items():
            if key == 'encoder.weight':
                renamed_state_dict['encoder.0.weight'] = value
            elif key == 'encoder.bias':
                renamed_state_dict['encoder.0.bias'] = value
            else:
                renamed_state_dict[key] = value
        return renamed_state_dict

    # Create an instance of the SAE model with the extracted parameters
    n_dict_components, input_size = sae_list['encoder.weight'].shape
    sae = SAE(input_size=input_size, n_dict_components=n_dict_components)

    # Rename the keys in the state dictionary
    renamed_sae_list = rename_keys(sae_list)

    # Load the state dictionary into the model instance
    sae.load_state_dict(renamed_sae_list)
    return sae

sae_local = load_sae(model_id, local_name).to(device)
sae_downstream = load_sae(model_id, downstream_name).to(device)

/var/folders/5w/vscxf8713fs_2w7g5f_rsb4c0000gn/T/ipykernel_27938/719909955.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sae_list = torch.load(ae_download_location)


In [4]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from einops import rearrange

def download_dataset(dataset_name, tokenizer, max_length=256, num_datapoints=None):
    if(num_datapoints):
        split_text = f"train[:{num_datapoints}]"
    else:
        split_text = "train"
    dataset = load_dataset(dataset_name, split=split_text).map(
        lambda x: tokenizer(x['text']),
        batched=True,
    ).filter(
        lambda x: len(x['input_ids']) > max_length
    ).map(
        lambda x: {'input_ids': x['input_ids'][:max_length]}
    )
    return dataset

dataset_name = "stas/openwebtext-10k"
max_seq_length = 40
print(f"Downloading {dataset_name}")
dataset = download_dataset(dataset_name, tokenizer=model.tokenizer, max_length=max_seq_length, num_datapoints=2000) # num_datapoints grabs all of them if None

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
# now run through the model & grab activations to get dead features
def get_dictionary_activations(model, dataset, cache_name, max_seq_length, autoencoder, batch_size=32):
    device = model.cfg.device
    num_features, d_model = autoencoder.encoder[0].weight.shape
    datapoints = dataset.num_rows
    dictionary_activations = torch.zeros((datapoints*max_seq_length, num_features))
    token_list = torch.zeros((datapoints*max_seq_length), dtype=torch.int64)
    with torch.no_grad(), dataset.formatted_as("pt"):
        dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
        for i, batch in enumerate(tqdm(dl)):
            batch = batch.to(device)
            token_list[i*batch_size*max_seq_length:(i+1)*batch_size*max_seq_length] = rearrange(batch, "b s -> (b s)")
            # with Trace(model, cache_name) as ret:
            #     _ = model(batch).logits
            #     internal_activations = ret.output
            #     # check if instance tuple
            #     if(isinstance(internal_activations, tuple)):
            #         internal_activations = internal_activations[0]
            _, cache = model.run_with_cache(batch)
            # print(cache)
            internal_activations = cache[cache_name]
            batched_neuron_activations = rearrange(internal_activations, "b s n -> (b s) n" )
            batched_dictionary_activations = autoencoder.encoder(batched_neuron_activations)
            dictionary_activations[i*batch_size*max_seq_length:(i+1)*batch_size*max_seq_length,:] = batched_dictionary_activations.cpu()
    return dictionary_activations, token_list

batch_size = 128
activation_name = f"blocks.{layer}.hook_resid_post"
nz_indices = []

for sae in [sae_local, sae_downstream]:
    dictionary_activations, _ = get_dictionary_activations(model, dataset, activation_name, max_seq_length, sae, batch_size=batch_size)
    nz_indices.append((dictionary_activations.sum(0) != 0).nonzero()[:, 0])

100%|██████████| 16/16 [00:57<00:00,  3.60s/it]


In [6]:
do_decoder = True

if(do_decoder):
    dec_local = sae_local.decoder.weight.data
    dec_downstream = sae_downstream.decoder.weight.data
else:
    dec_local = sae_local.encoder[0].weight.data.T
    dec_downstream = sae_downstream.encoder[0].weight.data.T

dec_local = dec_local / dec_local.norm(dim=0, keepdim=True)
dec_downstream = dec_downstream / dec_downstream.norm(dim=0, keepdim=True)

In [7]:
# Compute cos sim
cos_sim = torch.mm(dec_local.T, dec_downstream)

# Find pairs w/ cos sim > 0.999
threshold = 0.999
high_cos_sim_pairs = torch.nonzero(cos_sim > threshold)

In [8]:
len(high_cos_sim_pairs)

188

In [9]:
def get_feature_activations(sae, dataset, activation_name, feature_idx, batch_size=128):
    device = model.cfg.device
    activations = []
    
    with torch.no_grad(), dataset.formatted_as("pt"):
        dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
        for batch in tqdm(dl, desc=f"Processing feature {feature_idx}"):
            batch = batch.to(device)
            _, cache = model.run_with_cache(batch)
            internal_activations = cache[activation_name]
            batched_neuron_activations = rearrange(internal_activations, "b s n -> (b s) n")
            feature_activations = sae.encoder(batched_neuron_activations)[:, feature_idx]
            activations.append(feature_activations.cpu())
    
    return torch.cat(activations)

In [10]:
def find_activation_contexts(activations, token_list, top_k=5):
    top_indices = torch.argsort(activations, descending=True)[:top_k]
    contexts = [token_list[idx-2:idx+3].tolist() for idx in top_indices]
    return contexts

In [11]:
subset_examples = []

for local_idx, downstream_idx in tqdm(high_cos_sim_pairs[:10]):
    local_activations = get_feature_activations(sae_local, dataset, activation_name, local_idx)
    downstream_activations = get_feature_activations(sae_downstream, dataset, activation_name, downstream_idx)
    
    # Check if local feature activates on a subset of downstream feature
    # Count as active if above mean + std
    local_active = local_activations > local_activations.mean() + local_activations.std()
    downstream_active = downstream_activations > downstream_activations.mean() + downstream_activations.std()
    
    if local_active.sum() < downstream_active.sum() and (local_active & downstream_active).sum() / local_active.sum() > 0.2:
        local_contexts = find_activation_contexts(local_activations, token_list)
        downstream_contexts = find_activation_contexts(downstream_activations, token_list)
        
        subset_examples.append({
            'local_idx': local_idx.item(),
            'downstream_idx': downstream_idx.item(),
            'local_contexts': local_contexts,
            'downstream_contexts': downstream_contexts
        })

100%|██████████| 10/10 [16:31<00:00, 99.11s/it]


In [13]:
print(f"{len(subset_examples)} local feature that is a subset of downstream feature")

0 local feature that is a subset of downstream feature
